In [5]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import data as gdata
from IPython import display
from matplotlib import pyplot as plt
import time
import numpy as np

In [2]:
data = np.genfromtxt('D:/plantainz/OuO_DeepLearning/DataSet/airfoil_self_noise.dat',delimiter='\t')
data = (data - data.mean(axis = 0)) / data.std(axis = 0)
features,labels = np.array(data[:1500,:-1]),np.array(data[:1500,-1])

In [3]:
def init_states():
    v_w = nd.zeros((features.shape[1],1))
    v_b = nd.zeros(1)
    return (v_w,v_b)

In [ ]:
def f_2d(x1,x2): # 给Adagrad和下方RMSProp的示例函数
    return 0.1*x1**2 + 2*x2**2

# 图函数
---

In [6]:
def set_figsize(figsize=(3.5,2.5)):
    display.set_matplotlib_formats('svg')
    plt.rcParams['figure.figsize'] = figsize

In [4]:
# 用于画图``
def show_trace_2d(f,res):
    x1,x2 = zip(*res) # 如果res = [1,2,3,4]，那么就会变成 [(1,),(2,),(3,),(4,)] 
    set_figsize() # 设置矢量绘图方式 & 图的大小
    plt.plot(x1,x2,'-o',color = '#ff7f0e') # 
    
    x1 = np.arange(-5.5,1.0,0.1)
    x2 = np.arange(min(-3.0,min(x2)-1),max(1.0,max(x2+1),0.1))
    x1,x2 = np.meshgrid(x1,x2)
    
    plt.contour(x1,x2,f(x1,x2),colors = '#1f77b4')
    plt.xlabel('x1')
    plt.ylabel('x2')

# Momentum
---

In [9]:
def sgd_momentum(params,states,hyperparams):
    for p,v in zip(paramsms,states):
        v[:] = hyperparams['momentum'] * v + hyperparams['lr'] * p.grad
        p[:] -= v 

# Adagrad
---

In [ ]:
def adagrad(params,states,hyperparams):
    eps = 1e-6
    for p,s in zip(params,states):
        s[:] += p.grad.square()
        p[:] -= hyperparams['lr']*p.grad / (s+eps).sqrt()

# RMSProp
---

In [3]:
def rms_prop(params,states,hyperparams):
    gamma,eps = hyperparams['gamma'],1e-6
    for p,s in zip(params,states):
        s[:] = gamma*s + (1-gamma)*p.grad.square()
        p[:] -= hyperparams['lr'] * p.grad / (s+eps).sqrt()

# Adam
---

In [4]:
def init_adam_states():
    v_w,v_b = nd.zeros((features.shape[1],1),nd.zeros(1))
    s_w,s_b = nd.zeros((features.shape[1],1),nd.zeros(1))
    return ((v_w,v_b),(s_w,s_b))

In [ ]:
def adam(params,status,hyperparams):
    beta1,beta2,eps = 0.9,0.999,1e-6
    for p,(v,s) in zip(params,states):
        v[:] = beta1 * v + (1-beta1)*p.grad
        s[:] = beta2 * s + (1-beta2)*p.grad.square()
        
        v_bias_corr = v/(1-beta1 ** hyperparams['t'])
        s_bias_corr = s/(1-beta2 ** hyperparams['t'])
        
        p[:] -= hyperparams['lr'] * v_bias_corr / (s_bias_corr.sqrt() + eps)
    hyperparams['t'] += 1

# train()
---